In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
import bloqade
# from bloqade import KrylovKit

/home/ec2-user/anaconda3/envs/Braket/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [16]:
# Define constants
dim_pca = 10
Δ_max = 6.0
num_examples = 1000
num_test_examples = 100

In [17]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
test_dataset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)

In [18]:
# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

In [19]:
# Perform PCA on training data
pca = PCA(n_components=dim_pca)
x_train_pca = pca.fit_transform(train_dataset.data.numpy().reshape(-1, 28*28))
x_test_pca = pca.transform(test_dataset.data.numpy().reshape(-1, 28*28))

In [20]:
# Scale PCA values to feasible range of local detuning
x_train_pca = x_train_pca / np.max(np.abs(x_train_pca)) * Δ_max
x_test_pca = x_test_pca / np.max(np.abs(x_test_pca)) * Δ_max

In [21]:
x_test_pca[:, 1:num_examples]

array([[ 1.88088681, -0.1077645 , -0.78267303, ...,  1.18395488,
        -1.00375497, -0.46647276],
       [-2.40351538, -0.38411485,  1.00276769, ...,  0.58196908,
         0.37119781, -0.21784087],
       [-1.08367033,  0.1664487 ,  0.65421782, ..., -0.2765957 ,
         0.24811255, -0.3418475 ],
       ...,
       [ 1.50126648,  0.89318566, -1.04370463, ..., -0.88698179,
        -0.06128021, -2.01555388],
       [-0.27316387,  1.61689005, -0.63724006, ...,  0.41777847,
         0.67804019, -0.54465725],
       [-0.22766553,  1.77605246,  2.12734287, ..., -1.06862189,
         0.43011495,  0.13571932]])

In [22]:
# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(train_dataset.targets.numpy().reshape(-1, 1))
y_test = encoder.transform(test_dataset.targets.numpy().reshape(-1, 1))

In [23]:
y_test[:, 1:num_examples]

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
# Define quantum reservoir computing (QRC) layer
class DetuningLayer(nn.Module):
    def __init__(self, atoms, readouts, Ω, t_start, t_end, step):
        super(DetuningLayer, self).__init__()
        self.atoms = atoms
        self.readouts = readouts
        self.Ω = Ω
        self.t_start = t_start
        self.t_end = t_end
        self.step = step
    def forward(self, x):
    # Simulate quantum dynamics and compute readouts
    # have to use bloqade quantum
    # This part is not implemented in Python, as it requires a quantum simulator    
    # calculating steps
        self.atoms @ np.exp(-1j * h * (self.t_end - self.t_start))

## Defining a NN model

In [ ]:
# Define neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(dim_pca, 10)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return x
    # Train classical model using PCA features
    model_reg = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model_reg.parameters(), lr=0.01)
    for epoch in range(1000):
        for x, y in train_loader:
            x = x.view(-1, 28*28)
            x_pca = pca.transform(x.numpy())
            x_pca = torch.tensor(x_pca, dtype=torch.float32)
            y = torch.tensor(y, dtype=torch.long)
            optimizer.zero_grad()
            output = model_reg(x_pca)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
    # Train QRC model using quantum reservoir computing
    pre_layer = DetuningLayer(atoms, readouts, Ω, t_start, t_end, step)
    model_qrc = Net()
    for epoch in range(1000):
        for x, y in train_loader:
            x = x.view(-1, 28*28)
            x_pca = pca.transform(x.numpy())
